In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"adarshkl08","key":"#################"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c convolve-epoch1

 76% 50.0M/65.9M [00:00<00:00, 91.6MB/s]
100% 65.9M/65.9M [00:00<00:00, 99.9MB/s]


In [ ]:
!unzip /content/convolve-epoch1.zip -d /content/data

Archive:  /content/convolve-epoch1.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test.csv  
  inflating: /content/data/train.json  


In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_json("/content/data/train.json", orient="index").reset_index()
# renaming columns
df.rename(mapper={
        "index": "features",
        0: "label"
    },
    axis="columns",
    inplace=True)

In [ ]:
columns = [0, 1, 2, 3, 9]
df[columns] = df.features.str.split(n=4, expand=True)

In [ ]:
df[9] = df.apply(lambda x: x[9].split(" ", 1)[1] if x[2] != "-" else x[9], axis="columns")

In [ ]:
split = df[9].str.split(n=4, expand=True)
df[4] = split[0] + " " + split[1] + " " + split[2]
df[9] = split[3] + " " + split[4]

In [ ]:
df.rename(mapper={
        0: "ErrorId",
        2: "ErrorCode",
        3: "TimeStamp",
        9: "Details",
        4: "ErrorType"
},
    axis="columns",
    inplace=True)
df["Abnormal"] = df["label"].map(lambda x: 1 if x == "abnormal" else 0)
split = df["ErrorType"].str.split(expand=True)
df["ErrorLoc"] = split[0] + split[1]
df["ErrorType"] = split[2]

In [ ]:
df[["Code0", "Code1", "Code2", "Code3", "Code4"]] = df["ErrorCode"].str.split("-", expand=True)

In [ ]:
df["TimeStamp"] = pd.to_datetime(df["TimeStamp"], format="%Y-%m-%d-%H.%M.%S.%f")

In [ ]:
id_col = ["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8", "id9", "id10"]
df[id_col] = df["ErrorId"].str.split("", expand=True)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

In [ ]:
df = df[["id3", "id4", "id5", "id6", "id7", "id8", "id9", "id10", "Code0", "Code1", "Code2", "Code3", "Code4", "TimeStamp", "ErrorLoc", "ErrorType", "Details", "Abnormal"]]
df

,id3,id4,id5,id6,id7,id8,id9,id10,Code0,Code1,Code2,Code3,Code4,TimeStamp,ErrorLoc,ErrorType,Details,Abnormal
0,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.363779,RASKERNEL,INFO,instruction cache parity error corrected\n,0
1,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.527847,RASKERNEL,INFO,instruction cache parity error corrected\n,0
2,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.675872,RASKERNEL,INFO,instruction cache parity error corrected\n,0
3,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.823719,RASKERNEL,INFO,instruction cache parity error corrected\n,0
4,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.982731,RASKERNEL,INFO,instruction cache parity error corrected\n,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152654,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.386651,RASKERNEL,FATAL,data TLB error interrupt,1
4152655,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.558220,RASKERNEL,FATAL,data TLB error interrupt,1
4152656,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.675784,RASKERNEL,FATAL,data TLB error interrupt,1
4152657,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.777307,RASKERNEL,FATAL,data TLB error interrupt,1


In [ ]:
df.isna().any()
# df[df['Details'].isna()]  --36361
# df[df['Code1'].isna()]  --115573
# df[df['Code2'].isna()]  --116529
# df[df['Code3'].isna()]  --190406
# df[df['Code4'].isna()]  --190406

id3          False
id4          False
id5          False
id6          False
id7          False
id8          False
id9          False
id10         False
Code0        False
Code1         True
Code2         True
Code3         True
Code4         True
TimeStamp    False
ErrorLoc     False
ErrorType    False
Details       True
Abnormal     False
dtype: bool

In [ ]:
df[df['Details'].isna()]  #--36361
# df[df['Code1'].isna()]  --115573
# df[df['Code2'].isna()]  --116529
# df[df['Code3'].isna()]  --190406
# df[df['Code4'].isna()]  --190406

,id3,id4,id5,id6,id7,id8,id9,id10,Code0,Code1,Code2,Code3,Code4,TimeStamp,ErrorLoc,ErrorType,Details,Abnormal
31599,1,7,9,6,1,6,9,9,R11,M0,NB,C:J07,U11,2005-06-05 01:54:59.977568,RASKERNEL,FATAL,NaN,0
155818,1,8,1,4,7,5,7,7,R17,M0,N1,C:J03,U01,2005-06-07 05:32:57.199960,RASKERNEL,FATAL,NaN,0
187660,1,8,2,3,1,3,2,4,R33,M1,ND,C:J15,U11,2005-06-08 04:48:44.764098,RASKERNEL,FATAL,NaN,0
187770,1,8,2,3,1,3,3,7,R33,M1,ND,C:J15,U11,2005-06-08 04:48:57.129884,RASKERNEL,FATAL,NaN,0
187771,1,8,2,3,1,3,3,7,R33,M1,ND,C:J15,U11,2005-06-08 04:48:57.371441,RASKERNEL,FATAL,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979815,3,3,3,0,6,7,3,1,R67,M1,N9,C:J09,U01,2005-11-29 15:25:31.555725,RASKERNEL,FATAL,NaN,0
3979816,3,3,3,0,6,7,3,1,R67,M1,N9,C:J09,U01,2005-11-29 15:25:31.694633,RASKERNEL,FATAL,NaN,0
3979817,3,3,3,0,6,7,3,1,R67,M1,N9,C:J09,U01,2005-11-29 15:25:31.824847,RASKERNEL,FATAL,NaN,0
3979866,3,3,3,0,6,7,3,8,R67,M1,N9,C:J09,U01,2005-11-29 15:25:38.847677,RASKERNEL,FATAL,NaN,0


In [ ]:
df = df.dropna()
df

,id3,id4,id5,id6,id7,id8,id9,id10,Code0,Code1,Code2,Code3,Code4,TimeStamp,ErrorLoc,ErrorType,Details,Abnormal
0,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.363779,RASKERNEL,INFO,instruction cache parity error corrected\n,0
1,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.527847,RASKERNEL,INFO,instruction cache parity error corrected\n,0
2,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.675872,RASKERNEL,INFO,instruction cache parity error corrected\n,0
3,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.823719,RASKERNEL,INFO,instruction cache parity error corrected\n,0
4,1,7,8,3,8,5,7,0,R02,M1,N0,C:J12,U11,2005-06-03 15:42:50.982731,RASKERNEL,INFO,instruction cache parity error corrected\n,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152654,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.386651,RASKERNEL,FATAL,data TLB error interrupt,1
4152655,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.558220,RASKERNEL,FATAL,data TLB error interrupt,1
4152656,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.675784,RASKERNEL,FATAL,data TLB error interrupt,1
4152657,1,8,5,4,5,5,3,0,R30,M0,N9,C:J16,U01,2005-06-11 20:05:30.777307,RASKERNEL,FATAL,data TLB error interrupt,1


In [ ]:
df.isna().any()

id3          False
id4          False
id5          False
id6          False
id7          False
id8          False
id9          False
id10         False
Code0        False
Code1        False
Code2        False
Code3        False
Code4        False
TimeStamp    False
ErrorLoc     False
ErrorType    False
Details      False
Abnormal     False
dtype: bool

In [ ]:
# dummies = pd.get_dummies(df.Details)
# dummies.head()
pd.unique(df['Details'])

array(['instruction cache parity error corrected\n',
       'MidplaneSwitchController performing bit sparing on R27-M1-L3-U18-C bit 3\n',
       'generating core.304\n', ...,
       'ciod: failed to read message prefix on control stream (CioStream socket to 172.16.96.116:37976',
       'ciod: failed to read message prefix on control stream (CioStream socket to 172.16.96.116:37977',
       'machine check interrupt (bit=0x10): L2 DCU read error'],
      dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Creating Label Encoder instance.
labelencoder = LabelEncoder()

In [ ]:
labelencoder.fit(df['Details'])
df['Details'] = labelencoder.transform(df['Details'])

labelencoder.fit(df['ErrorType'])
df['ErrorType'] = labelencoder.transform(df['ErrorType'])

labelencoder.fit(df['ErrorLoc'])
df['ErrorLoc'] = labelencoder.transform(df['ErrorLoc'])

labelencoder.fit(df['Code0'])
df['Code0'] = labelencoder.transform(df['Code0'])

labelencoder.fit(df['Code1'])
df['Code1'] = labelencoder.transform(df['Code1'])


labelencoder.fit(df['Code2'])
df['Code2'] = labelencoder.transform(df['Code2'])

labelencoder.fit(df['Code3'])
df['Code3'] = labelencoder.transform(df['Code3'])

labelencoder.fit(df['Code4'])
df['Code4'] = labelencoder.transform(df['Code4'])

<ipython-input-19-63c26d9ddb6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Details'] = labelencoder.transform(df['Details'])
<ipython-input-19-63c26d9ddb6b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ErrorType'] = labelencoder.transform(df['ErrorType'])
<ipython-input-19-63c26d9ddb6b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
df

,id3,id4,id5,id6,id7,id8,id9,id10,Code0,Code1,Code2,Code3,Code4,TimeStamp,ErrorLoc,ErrorType,Details,Abnormal
0,1,7,8,3,8,5,7,0,2,1,3,10,2,2005-06-03 15:42:50.363779,1,1,244764,0
1,1,7,8,3,8,5,7,0,2,1,3,10,2,2005-06-03 15:42:50.527847,1,1,244764,0
2,1,7,8,3,8,5,7,0,2,1,3,10,2,2005-06-03 15:42:50.675872,1,1,244764,0
3,1,7,8,3,8,5,7,0,2,1,3,10,2,2005-06-03 15:42:50.823719,1,1,244764,0
4,1,7,8,3,8,5,7,0,2,1,3,10,2,2005-06-03 15:42:50.982731,1,1,244764,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152654,1,8,5,4,5,5,3,0,24,0,12,14,1,2005-06-11 20:05:30.386651,1,0,157363,1
4152655,1,8,5,4,5,5,3,0,24,0,12,14,1,2005-06-11 20:05:30.558220,1,0,157363,1
4152656,1,8,5,4,5,5,3,0,24,0,12,14,1,2005-06-11 20:05:30.675784,1,0,157363,1
4152657,1,8,5,4,5,5,3,0,24,0,12,14,1,2005-06-11 20:05:30.777307,1,0,157363,1


In [ ]:
# pd.unique(df['Details'])
n = len(pd.unique(df['Details']))

print("No.of.unique values :",
      n)

No.of.unique values : 246459


In [ ]:
from sklearn.model_selection import train_test_split

x = df[['id3',	'id4',	'id5',	'id6',	'id7',	'id8',	'id9',	'id10',	'Code0', 'Code1',	'Code2',	'Code3', 'Code4', 'ErrorLoc', 'ErrorType', 'Details']]
y = df['Abnormal']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

y_train

606959     0
3025514    0
1997149    0
1854530    0
1558654    0
          ..
2371612    0
3719325    0
2244148    0
2865741    0
2234174    0
Name: Abnormal, Length: 3140713, dtype: int64

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)

GaussianNB()

In [ ]:
model.score(x_test, y_test)

0.9843347822598414

# **Test CSV**

In [ ]:
test = pd.read_csv('/content/data/test.csv')
test1 = test
test

,ID,Log
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...
...,...,...
595295,595295,1118768234 2005.06.14 R20-M0-NA-C:J17-U11 200...
595296,595296,1118370947 2005.06.09 R15-M0-NA-C:J02-U01 200...
595297,595297,1118540242 2005.06.11 R30-M0-N9-C:J16-U01 200...
595298,595298,1118538947 2005.06.11 R30-M0-N9-C:J16-U01 200...


In [ ]:
test.columns

Index(['ID', ' Log'], dtype='object')

In [ ]:
columns = [0, 1, 2, 3, 9]
test[columns] = test[' Log'].str.split(n=4, expand=True)



test[9] = test.apply(lambda x: x[9].split(" ", 1)[1] if x[2] != "-" else x[9], axis="columns")
split = test[9].str.split(n=4, expand=True)
test[4] = split[0] + " " + split[1] + " " + split[2]
test[9] = split[3] + " " + split[4]
test.rename(mapper={
        0: "ErrorId",
        2: "ErrorCode",
        3: "TimeStamp",
        9: "Details",
        4: "ErrorType"
},
    axis="columns",
    inplace=True)
# test["Abnormal"] = test["label"].map(lambda x: 1 if x == "abnormal" else 0)

# test["ErrorType"].to_string()

test

,ID,Log,ErrorId,1,ErrorCode,TimeStamp,Details,ErrorType
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...,1124336301,2005.08.17,R13-M1-N8-C:J12-U01,2005-08-17-20.38.21.466368,rts: kernel terminated for reason 1001\n,RAS KERNEL FATAL
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...,1118553175,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-22.12.55.707149,data TLB error interrupt\n,RAS KERNEL FATAL
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...,1118536033,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-17.27.13.042387,data TLB error interrupt\n,RAS KERNEL FATAL
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...,1117992566,2005.06.05,R30-M1-N6-C:J03-U11,2005-06-05-10.29.26.943462,generating core.6463\n,RAS KERNEL INFO
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...,1118538965,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.16.05.049256,data TLB error interrupt\n,RAS KERNEL FATAL
...,...,...,...,...,...,...,...,...
595295,595295,1118768234 2005.06.14 R20-M0-NA-C:J17-U11 200...,1118768234,2005.06.14,R20-M0-NA-C:J17-U11,2005-06-14-09.57.14.709136,exception syndrome register: 0x00800000\n,RAS KERNEL FATAL
595296,595296,1118370947 2005.06.09 R15-M0-NA-C:J02-U01 200...,1118370947,2005.06.09,R15-M0-NA-C:J02-U01,2005-06-09-19.35.47.353971,generating core.10895\n,RAS KERNEL INFO
595297,595297,1118540242 2005.06.11 R30-M0-N9-C:J16-U01 200...,1118540242,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.37.22.743936,data TLB error interrupt\n,RAS KERNEL FATAL
595298,595298,1118538947 2005.06.11 R30-M0-N9-C:J16-U01 200...,1118538947,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.15.47.247097,data TLB error interrupt\n,RAS KERNEL FATAL


In [ ]:
split = test['ErrorType'].str.split(expand=True)

test["ErrorLoc"] = split[0] + split[1]
test["ErrorType"] = split[2]
test[["Code0", "Code1", "Code2", "Code3", "Code4"]] = test["ErrorCode"].str.split("-", expand=True)
test["TimeStamp"] = pd.to_datetime(test["TimeStamp"], format="%Y-%m-%d-%H.%M.%S.%f")
id_col = ["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8", "id9", "id10"]
test[id_col] = test["ErrorId"].str.split("", expand=True)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
#test = test[["id3", "id4", "id5", "id6", "id7", "id8", "id9", "id10", "Code0", "Code1", "Code2", "Code3", "Code4", "TimeStamp", "ErrorLoc", "ErrorType", "Details", "Abnormal"]]
test = test[["id3", "id4", "id5", "id6", "id7", "id8", "id9", "id10", "Code0", "Code1", "Code2", "Code3", "Code4", "TimeStamp", "ErrorLoc", "ErrorType", "Details"]]
test

,id3,id4,id5,id6,id7,id8,id9,id10,Code0,Code1,Code2,Code3,Code4,TimeStamp,ErrorLoc,ErrorType,Details
0,2,4,3,3,6,3,0,1,R13,M1,N8,C:J12,U01,2005-08-17 20:38:21.466368,RASKERNEL,FATAL,rts: kernel terminated for reason 1001\n
1,1,8,5,5,3,1,7,5,R30,M0,N9,C:J16,U01,2005-06-11 22:12:55.707149,RASKERNEL,FATAL,data TLB error interrupt\n
2,1,8,5,3,6,0,3,3,R30,M0,N9,C:J16,U01,2005-06-11 17:27:13.042387,RASKERNEL,FATAL,data TLB error interrupt\n
3,1,7,9,9,2,5,6,6,R30,M1,N6,C:J03,U11,2005-06-05 10:29:26.943462,RASKERNEL,INFO,generating core.6463\n
4,1,8,5,3,8,9,6,5,R30,M0,N9,C:J16,U01,2005-06-11 18:16:05.049256,RASKERNEL,FATAL,data TLB error interrupt\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595295,1,8,7,6,8,2,3,4,R20,M0,NA,C:J17,U11,2005-06-14 09:57:14.709136,RASKERNEL,FATAL,exception syndrome register: 0x00800000\n
595296,1,8,3,7,0,9,4,7,R15,M0,NA,C:J02,U01,2005-06-09 19:35:47.353971,RASKERNEL,INFO,generating core.10895\n
595297,1,8,5,4,0,2,4,2,R30,M0,N9,C:J16,U01,2005-06-11 18:37:22.743936,RASKERNEL,FATAL,data TLB error interrupt\n
595298,1,8,5,3,8,9,4,7,R30,M0,N9,C:J16,U01,2005-06-11 18:15:47.247097,RASKERNEL,FATAL,data TLB error interrupt\n


In [ ]:
labelencoder.fit(test['Details'])
test['Details'] = labelencoder.transform(test['Details'])

labelencoder.fit(test['ErrorType'])
test['ErrorType'] = labelencoder.transform(test['ErrorType'])

labelencoder.fit(test['ErrorLoc'])
test['ErrorLoc'] = labelencoder.transform(test['ErrorLoc'])

labelencoder.fit(test['Code0'])
test['Code0'] = labelencoder.transform(test['Code0'])

labelencoder.fit(test['Code1'])
test['Code1'] = labelencoder.transform(test['Code1'])


labelencoder.fit(test['Code2'])
test['Code2'] = labelencoder.transform(test['Code2'])

labelencoder.fit(test['Code3'])
test['Code3'] = labelencoder.transform(test['Code3'])

labelencoder.fit(test['Code4'])
test['Code4'] = labelencoder.transform(test['Code4'])

<ipython-input-29-72b27e2e21a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Details'] = labelencoder.transform(test['Details'])
<ipython-input-29-72b27e2e21a8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ErrorType'] = labelencoder.transform(test['ErrorType'])
<ipython-input-29-72b27e2e21a8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
xtest = test[['id3',	'id4',	'id5',	'id6',	'id7',	'id8',	'id9',	'id10',	'Code0', 'Code1',	'Code2',	'Code3', 'Code4', 'ErrorLoc', 'ErrorType', 'Details']]

pred = model.predict(xtest)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
pred = pd.DataFrame(pred,columns=[' Label'])

In [ ]:
sub = pd.concat([test1.ID,pred],axis=1)
sub.set_index('ID',inplace=True)

In [ ]:
sub

,Label
ID,
0,0
1,0
2,0
3,0
4,0
...,...
595295,0
595296,0
595297,0


In [ ]:
sub.to_csv(f"Convolve-Epoch.csv")